In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

test_seq = pd.read_csv('test_sequences.csv')
train_labels = pd.read_csv('train_labels.csv')
train_seq = pd.read_csv('train_sequences.csv')
val_labels = pd.read_csv('validation_labels.csv')
val_seq = pd.read_csv('validation_sequences.csv')

base_encoder = LabelEncoder()
base_encoder.fit(['A', 'U', 'G', 'C', 'N'])

def clean_and_encode_sequence(seq):
    clean_seq = seq.replace("-", "").replace("X", "N") 
    return np.array(base_encoder.transform(list(clean_seq)))  

train_seq["encoded_seq"] = train_seq["sequence"].apply(clean_and_encode_sequence)

train_labels["target_base"] = train_labels["ID"].apply(lambda x: "_".join(x.split("_")[:2]))

merged_data = train_seq.merge(
    train_labels, left_on="target_id", right_on="target_base"
)

merged_data.drop(columns=["target_base"], inplace=True)

merged_data.head()

X = merged_data[['sequence','temporal_cutoff','description']]
y = train_labels[['x_1', 'y_1', 'z_1']]

print(y.shape)
print(X.shape)

vectorizer = TfidfVectorizer()  # Initialize the TF-IDF vectorizer
X = vectorizer.fit_transform(X)  # Fit and transform the text data

X_train, X_test, y_train, y_test  = train_test_split(X,y, random_state=0)


model = LogisticRegression()
model.fit(X_train, y_train)

prediction  = model.predict(X_test)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))

(137095, 3)
(137095, 3)


ValueError: Found input variables with inconsistent numbers of samples: [3, 137095]